In [1]:
""" GUI and Image Handling """
from tkinter import *
from tkinter import ttk
import tkinter as tk
from tkinter import font
from PIL import ImageTk, Image, ImageDraw

""" Numerical and Scientific Processing """
import numpy as np
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

""" Visualization """
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
from matplotlib import cm

""" Native Packages """
import os
import itertools as it

""" Custom Subroutines """
from GUI_Helper_fcn import *


[NbConvertApp] Converting notebook GUI_Helper_fcn.ipynb to script
[NbConvertApp] Writing 60317 bytes to GUI_Helper_fcn.py


# Page 1

## Page 1 Functions

### Helper Functions

In [2]:
def get_curr_screen_geometry():
    """Retrieve the current display's dimensions

    :return: dims (tuple)
        Current display's width and height in units of pixels
    """
    temp = Tk()
    temp.update_idletasks()
    temp.attributes('-fullscreen', True)
    temp.state('iconic')
    geometry = temp.winfo_geometry()
    temp.destroy()
    dims = tuple([int(x) for x in geometry.split('+')[0].split('x')])
    return dims


def clip_im_z(mat):
    """Clip and min-max normalize the given matrix to range [-3, 3]. Operations are done on
    the copy of the original matrix.

    :param mat: A floating-point matrix (Numpy ndarray)
    :return: A floating-point matrix clipped and normalized to [-3, 3]. (Numpy ndarray).
    """
    temp = mat.copy()
    temp[temp > 3] = 3
    temp[temp < -3] = -3
    temp = (temp - (-3)) / 6
    return temp


def min_max_scale(a):
    """Helper function for min-max normalization

    :param a: Matrix to be normalization to range [0, 1]
    :return: Normalized matrix.
    """
    return (a - np.min(a)) / (np.max(a) - np.min(a))


def load_matrix(name):
    """Helper function for loading a matrix stored in specified file

    :param name: Path to the file storing matrix information.
    :return: A matrix (Numpy ndarray) is successfully loaded; else, None
    """
    try:
        with open(name, "rb") as f:
            a = np.load(f)
        return a
    except:
        print(f"File {name} Not Found!")
        return None


def load_image(name, w, h):
    """Load image with specified name and resized to specified shape

    :param name: Path to the image file - str
    :param w: desired image x dimension - int
    :param h: desired image y dimension - int
    :return: image - Pillow Image Object
        An Image object for Tkinter Interface with desired dimensions
    """
    image = Image.open(name)
    image = image.resize((w, h), Image.Resampling.LANCZOS)
    image = ImageTk.PhotoImage(image)
    return image


def updated_mask_edge(mat, save=False):
    """Run edge detection algorithms on the given matrix. May save resulting image to disk.

    :param mat: Matrix for edge detection (Numpy ndarray)
    :param save: value dictates whether edge is saved to disk.
    :return: Resulting edge contour lines
    """
    global STmap_file_prefix, STmap_orig_x_dim, STmap_orig_y_dim
    dpi = 1000
    return edgeDetection_GUI(mat, STmap_orig_x_dim / dpi, STmap_orig_y_dim / dpi,
                             dpi, STmap_file_prefix, FigDisplay=save)


def identify_pts_within_marked_region():
    """Identify points that are within the marked polygon.
    Matplotlib Path object (local variable called polygon) serves as the oracle for deciding whether a query point is
    or is not within the boundary of polygon. To speed up the query process, only points that are within the
    rectangular bounding box of polygon markers are queried.

    :return: A boolean matrix with size of canvas image/mask_matrix_canvas_size
        indicates which points are within the polygon defined by markers
    """
    global pg1_canvas, canvas_x_dim, canvas_y_dim, pg1_add_menu_options, pg1_add_curr_region
    mark = pg1_canvas.find_withtag("mark")
    pairs = [pg1_canvas.coords(mark[i])[:2] for i in range(len(mark))]
    pairs = np.array(pairs) - canvas_offset
    min_x, min_y = [int(i) for i in np.min(pairs, axis=0)]           # Define the polygon's bounding box
    max_x, max_y = [int(i) + 1 for i in np.max(pairs, axis=0)]
    points = [list(i) for i in it.product(np.arange(min_x, max_x),
                                          np.arange(min_y, max_y))]  # Mesh grid within the bounding box
    pairs[pairs == 0] = -1
    pairs[:, 0][pairs[:, 0] == int(canvas_x_dim - canvas_offset * 2 - 1)] = canvas_x_dim - canvas_offset * 2 + 1
    pairs[:, 1][pairs[:, 1] == int(canvas_y_dim - canvas_offset * 2 - 1)] = canvas_y_dim - canvas_offset * 2 + 1
    polygon = mpltPath.Path(pairs)                                   # Build a polygon surface, find all enclosed pts

    inside = polygon.contains_points(points, radius=1)               # Test if pts on mesh grid is inside or outside
    masks_s = np.reshape(inside, (max_x - min_x, max_y - min_y)).T   # Reshape the test results
    masks = np.full((int(canvas_y_dim - canvas_offset * 2), int(canvas_x_dim - canvas_offset * 2)), fill_value=False,
                    dtype=bool)                                      # All untested pts must be outside polygon
    masks[min_y:max_y, min_x:max_x] = masks_s
    field = np.full(masks.shape, fill_value=-1, dtype=float)         # Put boolean results back to larger boolean matrix
    field[masks == True] = pg1_add_menu_options[1][pg1_add_menu_options[0].index(pg1_add_curr_region.get())]
    field[masks == False] = -1
    return field


def interpolate(field):
    """Interpolate matrix from size of canvas image back to the dimensions of original mask file.
    To speed up the interpolation process, only points that are within the rectangular bounding box of polygon
    markers are tested.

    :param field: A boolean matrix with size of canvas image
    :return: A boolean matrix with size of original mask image
    """
    global pg1_canvas, canvas_x_dim, canvas_y_dim, STmap_orig_x_dim, STmap_orig_y_dim
    mark = pg1_canvas.find_withtag("mark")
    pairs = [pg1_canvas.coords(mark[i])[:2] for i in range(len(mark))]

    # Bounding box coordinates count in axis of canvas image (small)
    min_x, min_y = [i - canvas_offset for i in np.min(pairs, axis=0)]
    max_x, max_y = [i + 1 - canvas_offset for i in np.max(pairs, axis=0)]

    # Bounding box coordinates count in axis of original mask image (large)
    min_xc = int(np.floor((min_x * (STmap_orig_x_dim - 1) / (canvas_x_dim - 1 - canvas_offset * 2))))
    min_yc = int(np.floor((min_y * (STmap_orig_y_dim - 1) / (canvas_y_dim - 1 - canvas_offset * 2))))
    max_xc = int(np.ceil((max_x * (STmap_orig_x_dim - 1) / (canvas_x_dim - 1 - canvas_offset * 2))))
    max_yc = int(np.ceil((max_y * (STmap_orig_y_dim - 1) / (canvas_y_dim - 1 - canvas_offset * 2))))
    max_xc = min(max_xc, STmap_orig_x_dim - 1)
    max_yc = min(max_yc, STmap_orig_y_dim - 1)
    min_xc = max(min_xc, 0)
    min_yc = max(min_yc, 0)

    # The number of pts to query in x direction and y direction (large)
    cnt_xn = max_xc - min_xc + 1
    cnt_yn = max_yc - min_yc + 1

    # Bounding box coordinates count (large) in axis of canvas image (small)
    min_xn, max_xn = np.array([min_xc, max_xc]) * (canvas_x_dim - 1 - canvas_offset * 2) / (STmap_orig_x_dim - 1)
    min_yn, max_yn = np.array([min_yc, max_yc]) * (canvas_y_dim - 1 - canvas_offset * 2) / (STmap_orig_y_dim - 1)
    max_yn = min(max_yn, canvas_y_dim - canvas_offset * 2 - 1)
    max_xn = min(max_xn, canvas_x_dim - canvas_offset * 2 - 1)

    # Actual interpolation
    interp_func = RegularGridInterpolator((np.arange(int(canvas_y_dim - canvas_offset * 2)),
                                           np.arange(int(canvas_x_dim - canvas_offset * 2))),
                                          field, method="nearest")
    points = [list(i) for i in it.product(np.linspace(min_yn, max_yn, cnt_yn),
                                          np.linspace(min_xn, max_xn, cnt_xn))]
    output_s = interp_func(points)

    # Place interpolation results back to largely un-interpolated region
    output_s = np.reshape(output_s, (cnt_yn, cnt_xn))
    output = np.ones((STmap_orig_y_dim, STmap_orig_x_dim)) * -1
    output[min_yc:max_yc + 1, min_xc:max_xc + 1] = output_s

    return output


def build_modification_menu_options(mat):
    """Build modification menu for displaying and numerical processing.
    To assist with modification, users are allowed to paint region marked with 0 (a.k.a. erase currently
    indexed region) and a new number (a.k.a. add a new region).

    :param mat: An integer matrix where positive integers represent indexed regions
        all regions with no label have value of 0. (Numpy ndarray)
    :return: A list of list containing descriptions for display and integer values
        for downstream mapping between description and mask matrix.
    """
    options = set(mat.flatten().tolist())
    options.discard(0)
    options = list(options)
    descrip = [f"Add To Region {str(i)}" for i in options]
    extra_s = ["Erase An Existing Region", "Add A New Region"]
    if len(options):
        extra_n = [0, int(np.max(options) + 1)]
    else:
        extra_n = [0, 1]
    descrip.extend(extra_s)
    options.extend(extra_n)
    output = [descrip, options]
    return output


def build_deletion_menu_options(mat):
    """Build deletion menu for displaying and numerical processing
    To assist with modification, users are not only allowed to delete a specific region (by selecting
    the corresponding dropdown option) but also allowed to delete all currently indexed region (by selecting
    the option called 'Erase ALl Regions').

    :param mat: An integer matrix where positive integers represent indexed regions
        all regions with no label have value of 0. (Numpy ndarray)
    :return: A list of list containing descriptions for display and integer values
        for downstream mapping between description and mask matrix.
    """
    options = set(mat.flatten().tolist())
    options.discard(0)
    options = list(options)
    descrip = [f"Delete Region {str(i)}" for i in options]
    descrip.append("Erase All Regions")
    options.append(-2)
    output = [descrip, options]
    return output


def update_region_option_menu():
    """Update all menu entries for both the modification menu and the deletion menu on page 1.
    Helper functions build_modification_menu_options and build_deletion_menu_options are
    invoked to achieve modularization.

    :return: None
    """
    global pg1_add_menu_options, mask_matrix, pg1_add_curr_region, pg1_add_dropdown
    global pg1_del_menu_options, pg1_del_curr_region, pg1_del_dropdown
    pg1_add_menu_options = build_modification_menu_options(mask_matrix)
    pg1_del_menu_options = build_deletion_menu_options(mask_matrix)
    pg1_add_curr_region.set("Select Modification")
    pg1_del_curr_region.set("Delete Region(s) By Index")
    pg1_add_dropdown['menu'].delete(0, 'end')
    pg1_del_dropdown['menu'].delete(0, 'end')
    for choice in pg1_add_menu_options[0]:
        pg1_add_dropdown['menu'].add_command(label=choice, command=tk._setit(pg1_add_curr_region, choice))
    for choice in pg1_del_menu_options[0]:
        pg1_del_dropdown['menu'].add_command(label=choice, command=tk._setit(pg1_del_curr_region, choice))

        
def shrinking_matrix():
    """Down-sample the mask matrix to the size of canvas image.
    Constantly re-interpolate this matrix to ensure it is equivalent to the displayed mask image.

    :return: down-sampled mask integer matrix (Numpy ndarray)
    """
    global STmap_orig_y_dim, STmap_orig_x_dim, mask_matrix, canvas_y_dim, canvas_x_dim
    interp_func = RegularGridInterpolator((np.arange(STmap_orig_y_dim), np.arange(STmap_orig_x_dim)), mask_matrix,
                                          method="nearest")
    points = [list(i) for i in it.product(np.linspace(0, STmap_orig_y_dim - 1, int(canvas_y_dim - canvas_offset * 2)),
                                          np.linspace(0, STmap_orig_x_dim - 1, int(canvas_x_dim - canvas_offset * 2)))]
    shrunk_matrix = interp_func(points)
    shrunk_matrix = np.reshape(shrunk_matrix,
                               (int(canvas_y_dim - 2 * canvas_offset), int(canvas_x_dim - 2 * canvas_offset)))
    return shrunk_matrix


def render_first_page():
    """A major function that handles the rendering of page 1 canvas, specifically the canvas image gallery.
    The first image consists of edge contour overlayed onto clipped, normalized, z-score of STmap.
    The second image simply displays currently indexed region.
    The third image adds horizontal and vertical white strips (denoting light simulation time, observation time,
    and light simulation distance) onto the first image. In case where users do not specify value for any of the
    above three parameters, the third image is exactly the same as the first image.

    :return:
    """
    global iamge_pair, pg1_canvas_im_idx
    cancel_marks()
    pg1_canvas.delete("temp")

    # 1st Image
    dpi = 1000
    temp0 = clip_im_z(im_z)
    temp1 = edgeDetection_GUI(mask_matrix, STmap_orig_x_dim / dpi, STmap_orig_y_dim / dpi, dpi, STmap_file_prefix)
    orig_img = np.uint8(cm.jet(temp0) * 255)
    mask_clr = np.uint8(cm.binary(temp1) * 255)
    mask_clr[:, :, 3][temp1 == 0] = 0
    background = Image.fromarray(orig_img).resize(
        (int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2)), Image.Resampling.LANCZOS)
    foreground = Image.fromarray(mask_clr).resize(
        (int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2)), Image.Resampling.LANCZOS)
    background.paste(foreground, mask=foreground)
    pg1_canvas_im_gallery[0] = ImageTk.PhotoImage(background)

    # 2nd Image
    fig = plt.figure(frameon=False, figsize=(STmap_orig_x_dim / dpi, STmap_orig_y_dim / dpi), dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    dark_mask = np.uint8(cm.viridis(min_max_scale(mask_matrix)) * 255)
    dark_mask = Image.fromarray(dark_mask).resize(
        (int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2)), Image.Resampling.LANCZOS)
    pg1_canvas_im_gallery[1] = ImageTk.PhotoImage(dark_mask)

    # 3rd Image
    factor_y = STmap_orig_y_dim / (canvas_y_dim - canvas_offset * 2)
    factor_x = STmap_orig_x_dim / (canvas_x_dim - canvas_offset * 2)
    for sim_t in simulation_times:
        loc = int(sim_t * fps)
        orig_img[int(loc - factor_y * 1):int(loc + factor_y * 1), :, 0:3] = 255

        for segment in range(0, STmap_orig_x_dim, int(factor_x * 20)):
            loc = int((sim_t + observation_times) * fps)
            low_y = int(loc - factor_y * 1)
            upp_y = int(loc + factor_y * 1)
            low_x = int(segment - factor_x * 0)
            upp_x = int(segment + factor_x * 10)
            orig_img[low_y:upp_y, low_x:upp_x, :] = 255

    loc_x = int(simulation_location)
    orig_img[:, int(loc_x - factor_x * 1):int(loc_x + factor_x * 1), :] = 255

    background = Image.fromarray(orig_img).resize(
        (int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2)), Image.Resampling.LANCZOS)
    foreground = Image.fromarray(mask_clr).resize(
        (int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2)), Image.Resampling.LANCZOS)
    background.paste(foreground, mask=foreground)
    pg1_canvas_im_gallery[2] = ImageTk.PhotoImage(background)

    # Label indexed regions with their corresponding indices
    for region in pg1_add_menu_options[1][:-2]:
        vertices = np.argwhere(mask_matrix_canvas_size == region)
        if vertices.shape[0]:
            coord = [vertices[:, 1].mean(), vertices[:, 0].mean()]
            pg1_canvas.create_text(coord[0] + canvas_offset, coord[1] + canvas_offset, text=f'{region}',
                                   tags="temp", fill="#FFFFFF")

    # Display the 2nd plot
    pg1_canvas.itemconfig(pg1_canvas_txt_id, text="")
    pg1_canvas.itemconfig(image_id, image=pg1_canvas_im_gallery[1])
    pg1_canvas_im_idx = 1

### Event Handlers

In [3]:
def add_marker(event):
    """This function handles addition of marker upon user mouse clicking event.
    A marker is added if the click is done on page-1 canvas and a modification region has been chosen.

    :param event: A generic event object. In this case, a <ButtonPress>
    :return: None
    """
    global pg1_canvas
    if event.widget == pg1_canvas and str(preview_polygon['state']) == "normal":
        x, y = event.x, event.y
        # print(f'{x}, {y} - {canvas.canvasx(x)}, {canvas.canvasy(y)}')

        x = min(max(canvas_offset, x), canvas_x_dim - canvas_offset - 1)
        y = min(max(canvas_offset, y), canvas_y_dim - canvas_offset - 1)
        pg1_canvas.create_oval(x, y, x, y,
                               outline="#000000",
                               fill="#000000",
                               width=4, tags="mark")

        
def shift_left(event):
    """This function handles press of keyboard left key.
    Show the image (if exist) in front of the current displaying image in page-1 canvas gallery (a python list).
    If no such image exists, the current displaying image remains on the screen. All region indices will be
    re-colored to all black or all white in providing sufficient contrast to the background mask or STmap image.

    :param event: A generic event object. In this case, a <Left>.
    :return: None
    """
    global pg1_canvas_im_idx, pg1_canvas, pg1_add_menu_options, image_id, pg1_canvas_im_gallery

    pg1_canvas_im_idx = max(0, pg1_canvas_im_idx - 1)
    pg1_canvas.delete('temp')
    for region in pg1_add_menu_options[1][:-2]:
        vertices = np.argwhere(mask_matrix_canvas_size == region)
        if vertices.shape[0]:
            coord = [vertices[:, 1].mean(), vertices[:, 0].mean()]
            pg1_canvas.create_text(coord[0] + canvas_offset, coord[1] + canvas_offset, text=f'{region}',
                                   tags="temp", fill="#FFFFFF" if pg1_canvas_im_idx == 1 else "#000000")
    pg1_canvas.itemconfig(image_id, image=pg1_canvas_im_gallery[pg1_canvas_im_idx])


def shift_right(event):
    """This function handles press of keyboard right key.
    Show the image (if exist) behind the current displaying image in page-1 canvas gallery (a python list).
    If no such image exists, the current displaying image remains on the screen. All region indices will be
    re-colored to all black or all white in providing sufficient contrast to the background mask or STmap image.

    :param event: A generic event object. In this case, a <Right>.
    :return: None
    """
    global pg1_canvas_im_idx, pg1_canvas, pg1_add_menu_options, image_id, pg1_canvas_im_gallery

    pg1_canvas_im_idx = min(pg1_canvas_im_idx + 1, len(pg1_canvas_im_gallery) - 1)
    pg1_canvas.delete('temp')
    for region in pg1_add_menu_options[1][:-2]:
        vertices = np.argwhere(mask_matrix_canvas_size == region)
        if vertices.shape[0]:
            coord = [vertices[:, 1].mean(), vertices[:, 0].mean()]
            pg1_canvas.create_text(coord[0] + canvas_offset, coord[1] + canvas_offset, text=f'{region}',
                                   tags="temp", fill="#FFFFFF" if pg1_canvas_im_idx == 1 else "#000000")
    pg1_canvas.itemconfig(image_id, image=pg1_canvas_im_gallery[pg1_canvas_im_idx])

    
def display_indices(event):
    """This function handles the motion of mouse specified by user.
    A 3-d tuple of time, dist, and intensity will be shown if the mouse is hovering over the page-1 canvas
    image region and after all user-defined variables have been processed.

    :param event: A generic event object. In this case, a <Motion>.
    :return: None
    """
    global pg1_canvas, canvas_offset, canvas_y_dim, canvas_x_dim, STmap_orig_x_dim, STmap_orig_y_dim, fps, scale
    pg1_canvas.delete("index")
    widget = event.widget.winfo_containing(event.x_root, event.y_root)
    if widget != pg1_canvas:                                 # Must be within canvas widget
        return
    if str(confirm_modification["state"]) == "disabled":     # After all variables are initialized
        return
    if event.y < canvas_offset or event.x < canvas_offset:   # and be on the image (horizontally and vertically)
        return
    if event.y >= canvas_y_dim - canvas_offset or event.x >= canvas_x_dim - canvas_offset:
        return

    # Compute time (s), dist (cm), and z-score of STmap.
    time = (event.y - canvas_offset) / (canvas_y_dim - canvas_offset * 2) * STmap_orig_y_dim / fps
    dist = (event.x - canvas_offset) / (canvas_x_dim - canvas_offset * 2) * STmap_orig_x_dim / scale
    zval = im_z_canvas_size[int(event.y - canvas_offset), int(event.x - canvas_offset)]

    pg1_canvas.create_text(event.x, event.y, text=f"({time:.2f}, {dist:.2f}, {zval:.2f})", tags="index", fill="#000000")

### Main Routines

In [4]:
def handle_global_vars():
    """This function process user-defined input and initialize all global variables accordingly.

    :return: None
    """
    global STmap_file_prefix, fps, scale, STmap_orig_x_dim, STmap_orig_y_dim, mask_matrix, im_s, im_z, mask_c
    global mask_matrix_canvas_size, pg1_canvas, sample_feature_dict, pg1_canvas_im_gallery, im_z_canvas_size
    global simulation_times, observation_times, simulation_location

    sample_feature_dict = {}
    pg1_canvas.itemconfig(pg1_canvas_txt_id, text="Loading...")
    pg1_canvas.delete('temp')
    STmap_file_prefix = prefix_input_id.get()

    # Load STmap, z-score of STmap, and mask integer matrix
    im_s = load_matrix(f"raw_diameter_STmap_{STmap_file_prefix}.npy")
    im_z = load_matrix(f"zscore_STmap_{STmap_file_prefix}.npy")
    mask_c = load_matrix(f"mask_c_STmap_{STmap_file_prefix}.npy")

    if im_s is None or im_z is None or mask_c is None:
        pg1_canvas.itemconfig(pg1_canvas_txt_id, text="Bad file prefix.")
        return

    # Process fps and scale input
    try:
        fps = int(fps_input_id.get().strip())
        scale = float(scale_input_id.get().strip())
    except:
        pg1_canvas.itemconfig(pg1_canvas_txt_id, text="Improper fps or scale values.")
        return

    # Process parameters related to light simulation
    try:
        simulation_times = [float(i.strip()) for i in light_sim_time_input_id.get().split(',')]
        observation_times = float(light_sim_obs_time_input_id.get().strip())
        simulation_location = float(light_sim_dist_pix_input_id.get().strip())
    except:
        simulation_times = []
        observation_times = 0
        simulation_location = 0

    # Initialize all global variables used in the page 1.
    aspect = (shape(im_s)[1] / scale) / (shape(im_s)[0] / fps)
    setGlobal_GUI(fps, scale, STmap_file_prefix, aspect)
    pg1_canvas_im_gallery = [None, None, None]
    STmap_orig_y_dim, STmap_orig_x_dim = mask_c.shape
    mask_c = mask_c.astype(dtype=int)
    mask_matrix = mask_c
    mask_matrix_canvas_size = shrinking_matrix()
    update_region_option_menu()
    render_first_page()

    # Down-sample z-score of STmap
    interp_func = RegularGridInterpolator((np.arange(STmap_orig_y_dim), np.arange(STmap_orig_x_dim)), im_z,
                                          method="linear")
    points = [list(i) for i in it.product(np.linspace(0, STmap_orig_y_dim - 1, int(canvas_y_dim - canvas_offset * 2)),
                                          np.linspace(0, STmap_orig_x_dim - 1, int(canvas_x_dim - canvas_offset * 2)))]
    im_z_canvas_size = interp_func(points)
    im_z_canvas_size = np.reshape(im_z_canvas_size,
                                  (int(canvas_y_dim - 2 * canvas_offset), int(canvas_x_dim - 2 * canvas_offset)))

    # Enable various buttons on page 1
    pg1_add_dropdown.configure(state='normal')
    pg1_del_dropdown.configure(state='normal')
    confirm_modification['state'] = tk.NORMAL
    view_indexed_regions['state'] = tk.NORMAL
    reindex['state'] = tk.NORMAL
    load_alternative_mask['state'] = tk.NORMAL
    save_button['state'] = tk.NORMAL
    confirm_deletion['state'] = tk.NORMAL


def preview_regions_all():
    """Display all currently indexed regions' index.

    :return: None
    """
    global mask_matrix, STmap_file_prefix, STmap_orig_x_dim, STmap_orig_y_dim, pg1_canvas, mask_matrix_canvas_size
    pg1_canvas.delete("temp")

    pg1_canvas.itemconfig(image_id, image=pg1_canvas_im_gallery[1])
    cancel_marks()

    for region in pg1_add_menu_options[1][:-2]:
        vertices = np.argwhere(mask_matrix_canvas_size == region)
        if vertices.shape[0]:
            coord = [vertices[:, 1].mean(), vertices[:, 0].mean()]
            pg1_canvas.create_text(coord[0] + canvas_offset, coord[1] + canvas_offset, text=f'{region}',
                                   tags="temp", fill="#FFFFFF")


def load_mask():
    """This function loads an alternative mask specified by users.
    Upon loading the new mask, mask integer matrix, down-sampled mask matrix, menus,
    and page-1 image gallery all get updated accordingly.

    :return: None
    """
    global mask_c, pg1_canvas_im_gallery, STmap_orig_y_dim, STmap_orig_x_dim
    global mask_matrix_canvas_size, mask_matrix, pg1_canvas

    cancel_marks()
    pg1_canvas.delete("temp")
    if not os.path.exists(alternative_mask_id.get()):
        pg1_canvas.itemconfig(image_id, image=canvas_bg_image)
        pg1_canvas.itemconfig(pg1_canvas_txt_id, text="Specified Alternative Mask Not Found!")
        return

    alternative_mask_name = alternative_mask_id.get()
    if "csv" in alternative_mask_name:
        mask_c = pd.read_csv(alternative_mask_name, header=None, dtype=int).to_numpy()
    if "npy" in alternative_mask_name:
        with open(alternative_mask_name, "rb") as f:
            mask_c = np.load(f)

    pg1_canvas_im_gallery = [None, None, None]
    STmap_orig_y_dim, STmap_orig_x_dim = mask_c.shape
    mask_c = mask_c.astype(dtype=int)
    mask_matrix = mask_c
    mask_matrix_canvas_size = shrinking_matrix()
    update_region_option_menu()
    render_first_page()


def confirm_modification_region_selection():
    """Confirm the region to modify (mainly add) on pg1

    :return: None
    """
    global pg1_add_curr_region, pg1_dropdown_selection_txt, pg1_canvas, mask_matrix_canvas_size
    global pg1_add_menu_options, preview_polygon
    my_current_selection = pg1_add_curr_region.get()
    temp_txt = f"{my_current_selection} selected!\nPlease click on canvas to define the boundary of new region."
    pg1_dropdown_selection_txt.config(text=temp_txt, justify=tk.CENTER)

    preview_polygon['state'] = tk.NORMAL
    confirm_curr_polygon['state'] = tk.NORMAL
    cancel_curr_polygon['state'] = tk.NORMAL


def preview_region():
    """View region specified by the series of markers placed by users.
    The region will be filled with black paint.

    :return: None
    """
    global pg1_canvas
    mark = pg1_canvas.find_withtag("mark")
    coord_pairs = [pg1_canvas.coords(mark[i])[:2] for i in range(len(mark))]
    coords = [i for b in coord_pairs for i in b]
    if len(coords):
        pg1_canvas.create_polygon(*coords, activefill="#000000", fill="#000000", tags="region")


def cancel_marks():
    """Remove all drawing on page 1 canvas with tag "mark" and "region"

    :return: None
    """
    global pg1_canvas
    pg1_canvas.delete("mark")
    pg1_canvas.delete("region")


def update_region_modification():
    """This function handles the confirmation of region modifications.
    Mask integer matrix, down-sampled, mask matrix, all page-1 menus,
    and all images in page-1 gallery get updated in order listed here.

    :return: None
    """
    global pg1_add_curr_region, mask_matrix, STmap_file_prefix, image_id, pg1_add_menu_options
    global canvas_x_dim, canvas_y_dim, pg1_add_dropdown, pg1_frame, mask_matrix_canvas_size, pg1_canvas
    numeric_index = pg1_add_menu_options[1][pg1_add_menu_options[0].index(pg1_add_curr_region.get())]
    field = identify_pts_within_marked_region()      # Boolean mask indicates all pts within polygon
    interp = interpolate(field)                      # All pts within polygon on the original size
    mask_matrix[interp == numeric_index] = numeric_index
    _ = updated_mask_edge(mask_matrix)
    mask_matrix_canvas_size = shrinking_matrix()
    update_region_option_menu()
    render_first_page()

    preview_polygon['state'] = tk.DISABLED           # Disable all buttons that necessitate the selection of a region
    cancel_curr_polygon['state'] = tk.DISABLED
    confirm_curr_polygon['state'] = tk.DISABLED

    pg1_dropdown_selection_txt.config(text=f"Please Select A New Region If Need Be", justify=tk.CENTER)


def update_region_deletion():
    """This function handles the confirmation of region deletions.
    Similar to function 'update_region_modification', mask integer matrix, down-sampled, mask matrix,
    all page-1 menus, and all images in page-1 gallery get updated in order listed here.

    :return: None
    """
    global pg1_del_menu_options, pg1_del_curr_region, pg1_del_dropdown, mask_matrix, mask_matrix_canvas_size
    numeric_index = pg1_del_menu_options[1][pg1_del_menu_options[0].index(pg1_del_curr_region.get())]
    if numeric_index > 0:
        mask_matrix[mask_matrix == numeric_index] = 0
    elif numeric_index == -2:
        mask_matrix[:, :] = 0
    else:
        raise Exception("Sainty Check: Not Recognized Deletion Option.")
    edges = updated_mask_edge(mask_matrix)
    update_region_option_menu()
    mask_matrix_canvas_size = shrinking_matrix()
    render_first_page()


def reindex_regions():
    """Re-index all currently indexed regions on mask integer matrix.
    Helper function 'clean_mask_c_GUI' is used.

    :return: None
    """
    global mask_matrix, STmap_file_prefix, STmap_orig_x_dim, STmap_orig_y_dim, pg1_canvas, mask_matrix_canvas_size
    pg1_canvas.delete("temp")
    dpi = 1000
    mask_matrix = clean_mask_c_GUI(mask_matrix, STmap_orig_x_dim / dpi, STmap_orig_y_dim / dpi, dpi, STmap_file_prefix)
    update_region_option_menu()
    mask_matrix_canvas_size = shrinking_matrix()
    render_first_page()
    pg1_canvas_im_gallery[1] = load_image(f"{STmap_file_prefix}_reindexed.png", int(canvas_x_dim - canvas_offset * 2),
                                          int(canvas_y_dim - canvas_offset * 2))
    pg1_canvas.itemconfig(image_id, image=pg1_canvas_im_gallery[1])
    cancel_marks()

    for region in pg1_add_menu_options[1][:-2]:
        vertices = np.argwhere(mask_matrix_canvas_size == region)
        if vertices.shape[0]:
            coord = [vertices[:, 1].mean(), vertices[:, 0].mean()]
            pg1_canvas.create_text(coord[0] + canvas_offset, coord[1] + canvas_offset, text=f'{region}',
                                   tags="temp", fill="#FFFFFF")

    restart['state'] = tk.NORMAL
    pg1_next['state'] = tk.NORMAL


def save_figs():
    """Save the current mask matrix to disk (.csv). Save mask edge contour lines to image.

    :return: None
    """
    global STmap_file_prefix
    save_filename = save_file_path.get()
    if save_filename == "":
        np.savetxt(f"{STmap_file_prefix}_updated.csv", mask_matrix, delimiter=",", fmt="%u")
        save_status_label.config(text=f"Save Succeeded!", justify=tk.CENTER, anchor=tk.CENTER)
    else:
        try:
            np.savetxt(save_filename, mask_matrix, delimiter=",", fmt="%u")
            save_status_label.config(text=f"Save Succeeded!", justify=tk.CENTER, anchor=tk.CENTER)
        except Exception as e:
            save_status_label.config(text=f"Filename Invalid!\n{e}", justify=tk.CENTER, anchor=tk.CENTER)
    _ = updated_mask_edge(mask_matrix, save=True)


def reload_updated_images():
    """An express method of loading the updated mask integer matrix back to the GUI.

    :return: None
    """
    global mask_matrix, STmap_file_prefix, mask_matrix_canvas_size, pg1_canvas_im_gallery
    global canvas_x_dim, canvas_y_dim, image_id, restart, pg1_canvas
    cancel_marks()
    pg1_canvas.delete("temp")
    mask_matrix = pd.read_csv(f"{STmap_file_prefix}_updated.csv", header=None, dtype=int).to_numpy()
    edges = updated_mask_edge(mask_matrix)
    update_region_option_menu()
    mask_matrix_canvas_size = shrinking_matrix()
    render_first_page()

    restart['state'] = tk.DISABLED


def prepare_page2():
    """This helper function update page 2's menu/title and handles the actual lifting of frames.

    :return: None
    """
    global pg2_region_menu_options, mask_matrix
    pg2_region_menu_options = set(mask_matrix.flatten().tolist())
    pg2_region_menu_options.discard(0)
    pg2_region_menu_options = [f"Region {i}" for i in pg2_region_menu_options]
    pg2_curr_region.set("Select Region for Analysis")
    pg2_region_dropdown['menu'].delete(0, 'end')
    for choice in pg2_region_menu_options:
        pg2_region_dropdown['menu'].add_command(label=choice, command=tk._setit(pg2_curr_region, choice))
    root.title("Main Analysis")
    pg2_frame.lift()

## Page 1 Scripts

In [5]:
# Global Variables for page 1 and the other pages
STmap_file_prefix = ""
fps, scale = 0, 0
simulation_times = []
observation_times = 0
simulation_location = 0
pg1_canvas_im_gallery = []
pg1_canvas_im_idx = 0
STmap_orig_x_dim, STmap_orig_y_dim = None, None
canvas_offset = 20
mask_matrix, mask_matrix_canvas_size = None, None
im_s, im_z, mask_c = None, None, None
im_z_canvas_size = None
sample_feature_dict = {}

In [6]:
# Frame object must be confined within the current display
dim = get_curr_screen_geometry()

# Hard-coded maximum dimension of Canvas
canvas_x_dim, canvas_y_dim = min(dim[0] - 300, 800), min(dim[1] - 200, 800)

# Initialize software and features
root = Tk()
root.title("Mask Fine Tuning")
mfont = font.Font(family='Garamond', size=16, weight='bold')

# Initialize background placeholder image
canvas_bg_matrix = np.zeros((int(canvas_y_dim - canvas_offset * 2), int(canvas_x_dim - canvas_offset * 2), 3),
                            dtype=np.uint8)
canvas_bg_matrix[:, :, 0] = 104
canvas_bg_matrix[:, :, 1] = 172
canvas_bg_matrix[:, :, 2] = 229
canvas_bg_image = ImageTk.PhotoImage(Image.fromarray(canvas_bg_matrix, 'RGB'))

In [7]:
# Page 1
pg1_frame = ttk.Frame(root, width=dim[0], height=dim[1])
pg1_frame.grid(row=0, column=0, sticky=tk.N + tk.E + tk.S + tk.W)

# Render image on the frame
pg1_canvas = Canvas(pg1_frame, bg='#FFFFFF', width=canvas_x_dim, height=canvas_y_dim)
pg1_canvas.grid(row=0, column=0, rowspan=23, columnspan=2, sticky=tk.E + tk.W)
image_id = pg1_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image, anchor='nw')
pg1_canvas_txt_id = pg1_canvas.create_text(canvas_x_dim // 2, canvas_y_dim // 2,
                                           text="Welcome, Please Input File Prefix, Frame Per Second, and Scale.",
                                           justify=tk.CENTER)

# Initialize user-defined fields
prefix_input_id = ttk.Entry(pg1_frame)
fps_input_id = ttk.Entry(pg1_frame)
scale_input_id = ttk.Entry(pg1_frame)
light_sim_time_input_id = ttk.Entry(pg1_frame)
light_sim_obs_time_input_id = ttk.Entry(pg1_frame)
light_sim_dist_pix_input_id = ttk.Entry(pg1_frame)

# Place default user-defined fields
prefix_input_id.insert(0, "0628M2_2cm_pressure")
fps_input_id.insert(0, "10")
scale_input_id.insert(0, "331.6")
light_sim_time_input_id.insert(0, "60,240,420")
light_sim_obs_time_input_id.insert(0, "60")
light_sim_dist_pix_input_id.insert(0, "350")

# Hint users with field names
ttk.Label(pg1_frame, text="File Prefix").grid(row=0, column=2, sticky=tk.E + tk.W)
ttk.Label(pg1_frame, text="Frame Per Second").grid(row=1, column=2, sticky=tk.E + tk.W)
ttk.Label(pg1_frame, text="Pixel Distance Scale").grid(row=2, column=2, sticky=tk.E + tk.W)
ttk.Label(pg1_frame, text="Light Simulation Time").grid(row=3, column=2, sticky=tk.E + tk.W)
ttk.Label(pg1_frame, text="Light Simulation Observation Time").grid(row=4, column=2, sticky=tk.E + tk.W)
ttk.Label(pg1_frame, text="Light Simulation Distance Pixel").grid(row=5, column=2, sticky=tk.E + tk.W)

# Place user-defined fields on pg1_frame
prefix_input_id.grid(row=0, column=3, sticky=tk.E + tk.W)
fps_input_id.grid(row=1, column=3, sticky=tk.E + tk.W)
scale_input_id.grid(row=2, column=3, sticky=tk.E + tk.W)
light_sim_time_input_id.grid(row=3, column=3, sticky=tk.E + tk.W)
light_sim_obs_time_input_id.grid(row=4, column=3, sticky=tk.E + tk.W)
light_sim_dist_pix_input_id.grid(row=5, column=3, sticky=tk.E + tk.W)

# Confirm user-defined fields
confirm_global_var = ttk.Button(pg1_frame, text="Confirm All Global", command=handle_global_vars)
confirm_global_var.grid(row=6, column=2, columnspan=2, sticky=tk.E + tk.W)

# Show region index to current mask
view_indexed_regions = ttk.Button(pg1_frame, text="Preview Current Indexed Masks",
                                  command=preview_regions_all, state=tk.DISABLED)
view_indexed_regions.grid(row=7, column=2, columnspan=2, sticky=tk.E + tk.W)

# Or load a new mask
load_alternative_mask = ttk.Button(pg1_frame, text="Load Alternative Masks", command=load_mask, state=tk.DISABLED)
load_alternative_mask.grid(row=8, column=2, sticky=tk.E + tk.W)
alternative_mask_id = ttk.Entry(pg1_frame)
alternative_mask_id.grid(row=8, column=3, sticky=tk.E + tk.W)

In [8]:
# Separator -- Mask-level Operation vs Region-level operation
pg1_frame_upper_separator = ttk.Separator(pg1_frame)
pg1_frame_upper_separator.grid(row=9, column=2, sticky=tk.E + tk.W, columnspan=2)

# Render drop-down lists for region modification (mainly addition)
pg1_add_menu_options = [""]
pg1_add_curr_region = StringVar()
pg1_add_curr_region.set("Select Modification")
pg1_add_dropdown = OptionMenu(pg1_frame, pg1_add_curr_region, *pg1_add_menu_options)
pg1_add_dropdown.grid(row=10, column=2, columnspan=2, sticky=tk.E + tk.W)
pg1_add_dropdown.configure(state='disabled')

# Confirm region to modify next
confirm_modification = ttk.Button(pg1_frame, text="Confirm Modification Selection",
                                  command=confirm_modification_region_selection,
                                  state=tk.DISABLED)
confirm_modification.grid(row=11, column=2, columnspan=2, sticky=tk.E + tk.W)

# Display currently selected region
pg1_dropdown_selection_txt = ttk.Label(pg1_frame, text=" ")
pg1_dropdown_selection_txt.grid(row=12, column=2, columnspan=2, sticky=tk.E + tk.W)

# Create a temporary preview of polygon in commit
preview_polygon = ttk.Button(pg1_frame, text="Preview Selected Polygon", command=preview_region, state=tk.DISABLED)
preview_polygon.grid(row=13, column=2, columnspan=1, sticky=tk.E + tk.W)

# Cancel all set markers
cancel_curr_polygon = ttk.Button(pg1_frame, text="Cancel Previewed Polygon", command=cancel_marks, state=tk.DISABLED)
cancel_curr_polygon.grid(row=13, column=3, columnspan=1, sticky=tk.E + tk.W)

# Commit the polygon to be part of the mask
confirm_curr_polygon = ttk.Button(pg1_frame, text="Confirm Selected Polygon",
                                  command=update_region_modification, state=tk.DISABLED)
confirm_curr_polygon.grid(row=14, column=2, columnspan=2, sticky=tk.E + tk.W)

# Render drop-down lists for region deletion
pg1_del_menu_options = [""]
pg1_del_curr_region = StringVar()
pg1_del_curr_region.set("Delete Region(s) By Index")
pg1_del_dropdown = OptionMenu(pg1_frame, pg1_del_curr_region, *pg1_del_menu_options)
pg1_del_dropdown.grid(row=15, column=2, columnspan=2, sticky=tk.E + tk.W)
pg1_del_dropdown.configure(state='disabled')

# Confirm region to delete next
confirm_deletion = ttk.Button(pg1_frame, text="Confirm Deletion Selection",
                              command=update_region_deletion, state=tk.DISABLED)
confirm_deletion.grid(row=16, column=2, columnspan=2, sticky=tk.E + tk.W)

In [9]:
# Separator -- Region-level operation vs Mode selections
pg1_frame_lower_separator = ttk.Separator(pg1_frame)
pg1_frame_lower_separator.grid(row=17, column=2, sticky=tk.E + tk.W, columnspan=2)

# Re-label all indexed regions
reindex = ttk.Button(pg1_frame, text="Finish and Reindex All", command=reindex_regions, state=tk.DISABLED)
reindex.grid(row=18, column=2, columnspan=2, sticky=tk.E + tk.W)

# Save mask
save_button = ttk.Button(pg1_frame, text="Save Mask To File", command=save_figs, state=tk.DISABLED)
save_button.grid(row=19, column=2, sticky=tk.E + tk.W)

# Path to save mask
save_file_path = ttk.Entry(pg1_frame)
save_file_path.grid(row=19, column=3, sticky=tk.E + tk.W)

# Hint mask saving status
save_status_label = ttk.Label(pg1_frame, text=" ")
save_status_label.grid(row=20, column=2, columnspan=2, sticky=tk.E + tk.W)

# Go back to Edit Mode
restart = ttk.Button(pg1_frame, text="Go Back To Edit Mode", command=reload_updated_images, state=tk.DISABLED)
restart.grid(row=21, column=2, columnspan=2, sticky=tk.E + tk.W)

pg1_next = ttk.Button(pg1_frame, text="Proceed To Analysis", command=prepare_page2, state=tk.DISABLED)
pg1_next.grid(row=22, column=2, columnspan=2, sticky=tk.E + tk.W)

pg1_frame.columnconfigure(2, weight=1)
pg1_frame.columnconfigure(3, weight=1)

In [10]:
# Recognized input events
pg1_canvas.bind('<Motion>', display_indices)
root.bind('<ButtonPress>', add_marker)
root.bind('<KeyPress-Left>', shift_left)
root.bind('<KeyPress-Right>', shift_right)

'2725932863808shift_right'

# Page 2

## Page 2 Functions

### Helper Functions

### Event Handler

In [11]:
def display_indices_after_writing(event):
    """This function handles the displaying of a 3-d tuple <time (s), distance (cm), z-score of STmap>.
    This function is very similar to 'display_indices' defined for page 1. The tuple will not be shown
    if any of the conditional passes, see function inline comments for details.

    :param event: A generic event object. A '<Motion>'
    :return: None
    """
    global pg2_canvas, displaying_final_image, canvas_offset, canvas_y_dim, canvas_x_dim
    global STmap_orig_x_dim, STmap_orig_y_dim, fps, scale
    pg2_canvas.delete("index2")
    widget = event.widget.winfo_containing(event.x_root, event.y_root)
    if widget != pg2_canvas:                                # Must be within canvas2 widget
        return
    if not displaying_final_image:                          # Must be displaying the final image
        return
    if event.y < canvas_offset or event.x < canvas_offset:  # and be on the image (horizontally and vertically)
        return
    if event.y >= canvas_y_dim - canvas_offset or event.x >= canvas_x_dim - canvas_offset:
        return

    time = (event.y - canvas_offset) / (canvas_y_dim - canvas_offset * 2) * STmap_orig_y_dim / fps
    dist = (event.x - canvas_offset) / (canvas_x_dim - canvas_offset * 2) * STmap_orig_x_dim / scale
    zval = im_z_canvas_size[int(event.y - canvas_offset), int(event.x - canvas_offset)]

    pg2_canvas.create_text(event.x, event.y, text=f"({time:.2f}, {dist:.2f}, {zval:.2f})", tags="index2",
                           fill="#000000")

### Main Routines

In [12]:
def print_analyzed_regionIndex():
    """This function lists the index of analyzed region for page 2

    :return: None
    """
    if len(sample_feature_dict) == 0:
        return
    text = ""
    for key, value in sample_feature_dict.items():
        text = f"{text}{key[7:]};"
    pg2_label1.config(text=text, justify=tk.CENTER)


def write_all_to_results():
    """This function saves all extracted features to a file, and display a mask overlayed on z-score STmap with
    max-intensity of each region highlighted.
    Note: this function expects a file named 'analysis_result.xlsx' to already exist in the folder.

    :return: None
    """
    global displaying_final_image
    filename = "Analysis_result.xlsx"
    ExcelWorkbook = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = ExcelWorkbook
    # write each record to excel
    try:
        # write each record to excel
        for regionLabel, featuredict in sample_feature_dict.items():
            featuredf = pd.DataFrame()
            for k, v in featuredict.items():
                if isinstance(v, np.ndarray) and len(v.shape) > 1:
                    featuredf[k] = ''
                    featuredf.at[0, k] = list(v.flatten())
                else:
                    featuredf[k] = ''
                    featuredf.at[0, k] = v
            print(featuredict)

            sheetname = featuredf.loc[0]["class"]
            writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
            featuredf.to_excel(writer, sheet_name=sheetname, startrow=writer.book[sheetname].max_row, index=False,
                               header=False)
        writer.close()
        pg2_label2.config(text="Result File Updated. Please Proceed To Next Sample.", justify=tk.CENTER)
    except Exception as e:
        writer.close()
        pg2_label2.config(text=f"Failed To Save Results To Excel.\n{e}", justify=tk.CENTER)
        raise (e)

    # plot final image overlay
    dpi = 1000
    temp0 = clip_im_z(im_z)
    mask_curr_all = np.array(mask_matrix != 0).astype(float)
    img_blur_all = cv2.GaussianBlur(mask_curr_all, (3, 3), 0)
    edges_all = cv2.Canny(image=(img_blur_all * 255).astype(np.uint8),
                          threshold1=100, threshold2=150)  # Canny Edge Detection
    temp1 = edges_all
    orig_img = np.uint8(cm.jet(temp0) * 255)
    mask_clr = np.uint8(cm.binary(temp1) * 255)
    mask_clr[:, :, 3][temp1 == 0] = 0
    print(orig_img.shape)

    pairs = []
    for key, value in sample_feature_dict.items():
        coorlist = value["pixel coordinate of max z score contraction intensity"].tolist()
        for i in range(len(coorlist)):
            pairs.append(coorlist[i])

    for coor in pairs:
        x, y = coor[1], coor[0]
        orig_img[y - 15:y + 15, x - 8:x + 8, 0:3] = 0
        print(x, y)

    orig_img = Image.fromarray(orig_img)
    mask_clr = Image.fromarray(mask_clr)
    orig_img.paste(mask_clr, mask=mask_clr)
    orig_img.resize((300, 500), Image.Resampling.LANCZOS)
    orig_img.save(f"{STmap_file_prefix}_final.png")
    # load_image
    try:
        image = load_image(f"{STmap_file_prefix}_final.png", int(canvas_x_dim - canvas_offset * 2),
                           int(canvas_y_dim - canvas_offset * 2))
        pg2_canvas.itemconfig(pg2_canvas_image_id, image=image)
        pg2_canvas.image = image
        pg2_canvas.itemconfig(pg2_canvas_txt_id, text="")
        displaying_final_image = True
    except Exception as e:
        pg2_canvas.itemconfig(pg2_canvas_image_id, image=canvas_bg_image)
        pg2_canvas.itemconfig(pg2_canvas_txt_id, text=f"Fail To Display Final Result.\n{e}")


def wave_edge_analysis():
    """Run edge detection on a specific region and display the edge contour on page 2 canvas.

    :return: None
    """
    global STmap_file_prefix, pg2_curr_region, regionIndex, canvas_x_dim, canvas_y_dim
    global pg2_canvas, mask_matrix, displaying_final_image
    displaying_final_image = False
    try:
        regionIndex = int(pg2_curr_region.get().strip().split(' ')[1])
        edgeDetectionIndividual_GUI(mask_matrix, regionIndex, FigDisplay=True)
    except:
        pg2_canvas.itemconfig(pg2_canvas_image_id, image=canvas_bg_image)
        pg2_canvas.itemconfig(pg2_canvas_txt_id, text="Please Check If A Region is Selected.")

    try:
        image = load_image(f"{STmap_file_prefix}_region{regionIndex}_edge.png", int(canvas_x_dim - canvas_offset * 2),
                           int(canvas_y_dim - canvas_offset * 2))
        pg2_canvas.itemconfig(pg2_canvas_image_id, image=image)
        pg2_canvas.image = image
        pg2_canvas.itemconfig(pg2_canvas_txt_id, text="")
    except Exception as e:
        pg2_canvas.itemconfig(pg2_canvas_image_id, image=canvas_bg_image)
        pg2_canvas.itemconfig(pg2_canvas_txt_id, text=f"Fail To Detect Wave Edge.\n{e}")


def enable_button_to_page3(whatev):
    """Upon selecting a contraction type, allow users to proceed to page 3 (propagation or ripple version)

    :param whatev: None
    :return: None
    """
    global contractionType
    pg2_next['state'] = tk.NORMAL
    contractionType = pg2_curr_contraction.get()


def go_back_to_page1():
    """Update GUI title according to the currently visible frame.
    Do the actual lifting of page 1.

    :return: None
    """
    root.title("Mask Fine Tuning")
    pg1_frame.lift()


def advance_beyond_page2():
    """Jump to page 3 propagation or page 3 ripple depending on user's decision of region contraction type.
    Also, update GUI title accordingly.

    :return: None
    """
    if pg2_curr_contraction.get() == "Ripple":
        root.title("Ripple Analysis")
        pg3_ripple_frame.lift()
    else:
        root.title("Propagation Analysis Step 1")
        pg3_prop_frame.lift()

## Page 2 Scripts

In [13]:
contractionType = ""
regionIndex = -1
displaying_final_image = False

In [14]:
pg2_frame = ttk.Frame(root, width=dim[0], height=dim[1])
pg2_frame.grid(row=0, column=0, sticky=tk.N + tk.E + tk.S + tk.W)

# Page 2 Canvas
pg2_canvas = Canvas(pg2_frame, bg='#FFFFFF', width=canvas_x_dim, height=canvas_y_dim)
pg2_canvas.grid(row=0, column=0, rowspan=10, columnspan=2)
pg2_canvas_image_id = pg2_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image, anchor='nw')
pg2_canvas_txt_id = pg2_canvas.create_text(canvas_x_dim // 2, canvas_y_dim // 2, width=canvas_x_dim,
                                           text="Wave Edge Plot Will Be Displayed Here!", justify=tk.CENTER)

# Button to print analyzed regions
already_analyzed_region = ttk.Button(pg2_frame, text="Display Already Analyzed Regions",
                                     command=print_analyzed_regionIndex)
already_analyzed_region.grid(row=0, column=2, columnspan=2, sticky=tk.E + tk.W)

# Placeholder for the print of already analyzed regions
pg2_label1 = ttk.Label(pg2_frame, text="No region has been analyzed.", justify=tk.CENTER, anchor=tk.CENTER)
pg2_label1.grid(row=1, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to finish all and write to results
write_result = ttk.Button(pg2_frame, text="Finish Analysis And Write All to File", command=write_all_to_results)
write_result.grid(row=2, column=2, columnspan=2, sticky=tk.E + tk.W)

# Placeholder for the save result confirmation
pg2_label2 = ttk.Label(pg2_frame, text="Analysis In Progress.", justify=tk.CENTER, anchor=tk.CENTER)
pg2_label2.grid(row=3, column=2, columnspan=2, sticky=tk.E + tk.W)

In [15]:
# Separate
pg2_separator = ttk.Separator(pg2_frame)
pg2_separator.grid(row=4, column=2, sticky=tk.E + tk.W, columnspan=2)

# Menu for Selecting Region
pg2_region_menu_options = [""]
pg2_curr_region = StringVar()
pg2_curr_region.set("Select New Region for Analysis")
pg2_region_dropdown = OptionMenu(pg2_frame, pg2_curr_region, *pg2_region_menu_options)
pg2_region_dropdown.grid(row=5, column=2, sticky=tk.E + tk.W)

run_wave_edge_button = ttk.Button(pg2_frame, text="Detect Wave Edge", command=wave_edge_analysis)
run_wave_edge_button.grid(row=5, column=3, sticky=tk.E + tk.W)

# Instruction/Reminder for Contraction Type
pg2_label3 = ttk.Label(pg2_frame, text="Please Select A Preliminary Contraction Type.",
                       justify=tk.CENTER, anchor=tk.CENTER)
pg2_label3.grid(row=6, column=2, columnspan=2, sticky=tk.E + tk.W)

# Menu for Selecting Contraction Type (Coarse)
pg2_contraction_menu_options = ['Propagation', 'Ripple']
pg2_curr_contraction = StringVar()
pg2_curr_contraction.set("Contraction Types")
pg2_contraction_dropdown = OptionMenu(pg2_frame, pg2_curr_contraction, *pg2_contraction_menu_options,
                                      command=enable_button_to_page3)
pg2_contraction_dropdown.grid(row=7, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to back to Page 1
pg2_prev = ttk.Button(pg2_frame, text="Back To Mask Fine Tuning", command=go_back_to_page1)
pg2_prev.grid(row=8, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to go to Page 3 (Branching to Propagation or Ripple)
pg2_next = ttk.Button(pg2_frame, text="Proceed To Next Page",
                      command=advance_beyond_page2,
                      state=tk.DISABLED)
pg2_next.grid(row=9, column=2, columnspan=2, sticky=tk.E + tk.W)

pg2_frame.columnconfigure(2, weight=1)
pg2_frame.columnconfigure(3, weight=1)

In [16]:
pg2_canvas.bind('<Motion>', display_indices_after_writing)

'2725933216576display_indices_after_writing'

# Page 3 Propagation

## Page 3 Propagation Functions

### Helper Functions

### Event Handlers

### Main Routines

In [17]:
def wave_edge_prop_detection():
    """Run helper function 'getpropagationDir_GUI' to obtain automated prediction
    on the type of propagation direction for the selected region. Display the results
    if the helper succeeds.

    :return: None
    """
    global propagation_dir

    try:
        propagation_dir = getpropagationDir_GUI(mask_matrix, regionIndex, FigDisplay=True)
        pg3_prop_label1.config(text=f"Predicted Direction Is - <{propagation_dir}>", justify=tk.CENTER)

        pg3_prop_canvas.itemconfig(pg3_prop_canvas_txt_id, text="")
        image = load_image(f"{STmap_file_prefix}_region{regionIndex}_repivoted.png",
                           int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2))
        pg3_prop_canvas.itemconfig(pg3_prop_canvas_image_id, image=image)
        pg3_prop_canvas.image = image

        pg3_prop_label2.config(text=f"Please Select Final Decision\nfor Propagation Direction", justify=tk.CENTER)
        pg3_prop_dropdown.configure(state='normal')

    except Exception as e:
        pg3_prop_canvas.itemconfig(pg3_prop_canvas_txt_id, text=f"Fail To Predict Propagation Direction\n{e}")


def enable_button_to_page4(whatev):
    """Upon users select a propagation direction (manually correct algorithm prediction, allow users to
    proceed to the rest of propagation analysis.

    :param whatev: None
    :return: None
    """
    global propagation_dir
    propagation_dir = str(pg3_prop_curr_dir.get()).lower()
    pg3_prop_next['state'] = tk.NORMAL


def go_back_to_page2():
    """Allow going back to the page 2. Alter title for GUI and bring page 2 to the top of stack.

    :return: None
    """
    root.title("Main Analysis")
    pg2_frame.lift()


def proceed_to_page4():
    """Allow users to proceed to page 4. Alter title and bring page 4 to top.

    :return: None
    """
    root.title("Propagation Analysis Step 2")
    pg4_frame.lift()

## Page 3 Propagation Scripts

In [18]:
propagation_dir = ""

In [19]:
pg3_prop_frame = ttk.Frame(root, width=dim[0], height=dim[1])
pg3_prop_frame.grid(row=0, column=0, sticky=tk.N + tk.E + tk.S + tk.W)

pg3_prop_canvas = Canvas(pg3_prop_frame, bg='#FFFFFF', width=canvas_x_dim, height=canvas_y_dim)
pg3_prop_canvas.grid(row=0, column=0, rowspan=7, columnspan=2)
pg3_prop_canvas_image_id = pg3_prop_canvas.create_image(canvas_offset, canvas_offset,
                                                        image=canvas_bg_image, anchor='nw')
pg3_prop_canvas_txt_id = pg3_prop_canvas.create_text(canvas_x_dim // 2, canvas_y_dim // 2, width=canvas_x_dim,
                                                     text="Wave Edge Detection Re-pivoted Image Goes Here!",
                                                     justify=tk.CENTER)

# To Run Propagation Prediction
pg3_predict_prop = ttk.Button(pg3_prop_frame, text="Predict Propagation Direction", command=wave_edge_prop_detection)
pg3_predict_prop.grid(row=0, column=2, columnspan=2, sticky=tk.E + tk.W)

# Placeholder for Predicted Propagation Type
pg3_prop_label1 = ttk.Label(pg3_prop_frame, text=" ", justify=tk.CENTER, anchor=tk.CENTER)
pg3_prop_label1.grid(row=1, column=2, columnspan=2, sticky=tk.E + tk.W)

In [20]:
# Separate
pg3_prop_separator = ttk.Separator(pg3_prop_frame)
pg3_prop_separator.grid(row=2, column=2, sticky=tk.E + tk.W, columnspan=2)

# Placeholder for Predicted Propagation Direction Type
pg3_prop_label2 = ttk.Label(pg3_prop_frame, text=" ", justify=tk.CENTER, anchor=tk.CENTER)
pg3_prop_label2.grid(row=3, column=2, columnspan=2, sticky=tk.E + tk.W)

# Menu for Final Decided Propagation Direction
pg3_prop_menu_options = ['Anterograde Propagation', 'Retrograde Propagation', 'Bidirectional Propagation']
pg3_prop_curr_dir = StringVar()
pg3_prop_curr_dir.set("Propagation Direction Types")
pg3_prop_dropdown = OptionMenu(pg3_prop_frame, pg3_prop_curr_dir, *pg3_prop_menu_options,
                               command=enable_button_to_page4)
pg3_prop_dropdown.grid(row=4, column=2, columnspan=2, sticky=tk.E + tk.W)
pg3_prop_dropdown.configure(state='disabled')

# Button to back to Page 2
pg3_prop_prev = ttk.Button(pg3_prop_frame, text="Back To Previous Page", command=go_back_to_page2)
pg3_prop_prev.grid(row=5, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to Page 4
pg3_prop_next = ttk.Button(pg3_prop_frame, text="Proceed To Next Page", command=proceed_to_page4,
                           state=tk.DISABLED)
pg3_prop_next.grid(row=6, column=2, columnspan=2, sticky=tk.E + tk.W)

pg3_prop_frame.columnconfigure(2, weight=1)
pg3_prop_frame.columnconfigure(3, weight=1)

# Page 3 Ripple

## Page 3 Ripple Functions

### Helper Functions

### Event Handlers

In [21]:
def on_mousewheel(event):
    """Handle scroll for page 3 ripple canvas

    :param event: A generic event object. A <MouseWheel>
    :return: None
    """
    pg3_ripple_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

### Main Routines

In [22]:
def ripple_feature():
    """This function handles the feature extraction for region classified as ripple contraction.
    A bounding box around the region is fed to helper function 'rippleregionFeature' and resulting
    descriptions are displayed on screen.

    :return: None
    """
    global feature_descriptions_ripple, rippletemp, page3_bg, pg3_ripple_image_id
    try:
        # Selecting bounding box for helper function
        pts_of_region = np.argwhere(mask_matrix == regionIndex)
        ymin, xmin = np.min(pts_of_region, axis=0)
        ymax, xmax = np.max(pts_of_region, axis=0)
        rippletemp = rippleregionFeature(mask_matrix, im_s, im_z, regionIndex, xmin / scale, xmax / scale, ymin / fps,
                                         ymax / fps)

        # Alter texts for pretty display with scrollable display on canvas (if needed)
        beautify = [f'{k} : {v}\n' for k, v in rippletemp.items()]
        feature_descriptions_ripple = '\n'.join(beautify)
        pg3_ripple_canvas.itemconfig(pg3_ripple_txt_id, text=feature_descriptions_ripple)
        corners = pg3_ripple_canvas.bbox(pg3_ripple_txt_id)
        bbox_height = corners[3] - corners[1]
        if bbox_height > canvas_y_dim - canvas_offset * 2:
            pg3_ripple_canvas.delete(pg3_ripple_image_id)
            total_height = bbox_height + 60
            page3_bg = ImageTk.PhotoImage(
                Image.fromarray(canvas_bg_matrix).resize((canvas_x_dim, total_height), Image.Resampling.LANCZOS))
            pg3_ripple_image_id = pg3_ripple_canvas.create_image(canvas_offset, canvas_offset - (
                        total_height - (canvas_y_dim - canvas_offset * 2)) / 2,
                                                                 image=page3_bg, anchor='nw')
            pg3_ripple_canvas.image = page3_bg
            pg3_ripple_canvas.itemconfig(pg3_ripple_txt_id, text=feature_descriptions_ripple)
            pg3_ripple_canvas.tag_raise(pg3_ripple_txt_id)

    except Exception as e:
        pg3_ripple_canvas.itemconfig(pg3_ripple_txt_id, text=f"Fail To Extract Ripple Features\n{e}")


def save_features_ripple():
    """Save extracted ripple features to global var, for future saving to disk

    :return: None
    """
    global sample_feature_dict, rippletemp
    sample_feature_dict[f"region_{regionIndex}"] = rippletemp


def return_page2():
    """Reset all buttons, texts, images, variables, states upon returning to Main Analysis page

    :return: None
    """
    global contractionType, regionIndex, propagation_dir, feature_descriptions_ripple, rippletemp, pg4_canvas_image_id
    global frame4_gallery, frame4_current_index, feature_descriptions, proptemp, pg3_ripple_image_id
    root.title("Main Analysis")
    pg2_canvas.itemconfig(pg2_canvas_image_id, image=canvas_bg_image)
    pg2_canvas.itemconfig(pg2_canvas_txt_id, text="Wave Edge Plot Will Be Displayed Here!", justify=tk.CENTER)
    pg2_next['state'] = tk.DISABLED
    pg2_curr_region.set("Select New Region for Analysis")
    pg2_curr_contraction.set("Contraction Types")
    contractionType = ""
    regionIndex = -1

    propagation_dir = ""
    pg3_prop_label1.config(text=f" ", justify=tk.CENTER)
    pg3_prop_label2.config(text=f" ", justify=tk.CENTER)
    pg3_prop_canvas.itemconfig(pg3_prop_canvas_txt_id, text="Wave Edge Detection Repivoted Image Goes Here!",
                               justify=tk.CENTER)
    pg3_prop_canvas.itemconfig(pg3_prop_canvas_image_id, image=canvas_bg_image)
    pg3_prop_curr_dir.set("Propagation Direction Types")
    pg3_prop_dropdown.configure(state='disabled')
    pg3_prop_next['state'] = tk.DISABLED

    feature_descriptions_ripple = ""
    pg3_ripple_canvas.itemconfig(pg3_ripple_txt_id, text="Ripple!", justify=tk.CENTER)
    pg3_ripple_canvas.delete(pg3_ripple_image_id)
    pg3_ripple_image_id = pg3_ripple_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image,
                                                         anchor='nw')
    pg3_ripple_canvas.tag_raise(pg3_ripple_txt_id)
    rippletemp = ""

    frame4_gallery = []
    frame4_current_index = 0
    feature_descriptions = ""
    proptemp = ""
    pg4_canvas.itemconfig(pg4_canvas_txt_id, text="Wave Fronts Analysis Plots Go Here.", justify=tk.CENTER)
    pg4_canvas.delete(pg4_canvas_image_id)
    pg4_canvas_image_id = pg4_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image, anchor='nw')
    pg4_canvas.tag_raise(pg4_canvas_txt_id)
    pg4_curr_contraction.set("Propagation Direction Types")
    pg4_curr_detection_mode.set("Detection Mode Types")
    ati_in.delete(0, 'end')
    ats_in.delete(0, 'end')
    rti_in.delete(0, 'end')
    rts_in.delete(0, 'end')
    ati_in.insert(0, "0.5")
    ats_in.insert(0, "5000")
    rti_in.insert(0, "0.5")
    rts_in.insert(0, "5000")
    pg4_label.config(text=f"")

    pg2_frame.lift()

## Page 3 Ripple Scripts

In [23]:
feature_descriptions_ripple = ""
rippletemp = ""
page3_bg = None

In [24]:
pg3_ripple_frame = ttk.Frame(root, width=dim[0], height=dim[1])
pg3_ripple_frame.grid(row=0, column=0, sticky=tk.N + tk.E + tk.S + tk.W)

pg3_ripple_canvas = Canvas(pg3_ripple_frame, bg='#FFFFFF', width=canvas_x_dim, height=canvas_y_dim)
pg3_ripple_canvas.grid(row=0, column=0, rowspan=3, columnspan=2)
pg3_ripple_image_id = pg3_ripple_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image, anchor='nw')
pg3_ripple_txt_id = pg3_ripple_canvas.create_text(canvas_x_dim // 2, canvas_y_dim // 2, width=canvas_x_dim,
                                                  text="Ripple!", justify=tk.CENTER)

# Button to run feature extraction
pg3_ripple_ft_extract = ttk.Button(pg3_ripple_frame, text="Run Feature Extraction for Ripples", command=ripple_feature)
pg3_ripple_ft_extract.grid(row=0, column=2, columnspan=2, sticky=tk.E + tk.W)

# Save Features
pg3_ripple_save_ft = ttk.Button(pg3_ripple_frame, text="Save Extracted Features", command=save_features_ripple)
pg3_ripple_save_ft.grid(row=1, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to back to Page 2
pg3_ripple_done = ttk.Button(pg3_ripple_frame, text="Finish And Go To Analysis Main Page", command=return_page2)
pg3_ripple_done.grid(row=2, column=2, columnspan=2, sticky=tk.E + tk.W)

pg3_ripple_frame.columnconfigure(2, weight=1)
pg3_ripple_frame.columnconfigure(3, weight=1)

In [25]:
pg3_ripple_canvas.bind("<MouseWheel>", on_mousewheel)

'2725932907200on_mousewheel'

# Page 4

## Page 4 Functions

### Helper Functions

### Event Handlers

In [26]:
def on_mousewheel(event):
    """Bind mouse wheel scrolling to the y-dimension of the page-4 canvas display.

    :param event: A generic event object. A <MouseWheel>
    :return: None
    """
    pg4_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

### Main Routines

In [27]:
def update_contraction_type(whatev):
    """Update contractionType global with finer options

    :param whatev: None
    :return: None
    """
    global contractionType
    contractionType = str(pg4_curr_contraction.get()).lower()


def run_feature_extraction():
    """Run helper function propregionFeature_GUI to obtain analysis results
    Display resulting images and analysis descriptions using the page 4 canvas
    gallery. Pages can be reached via left and right buttons shown on bottom
    on the page 4. The description page can be viewed in its entirety via scrolling.

    :return: None
    """
    global feature_descriptions, frame4_current_index, frame4_gallery, proptemp, pg4_canvas_image_id, page4_bg

    frame4_gallery = []

    try:
        proptemp = propregionFeature_GUI(mask_matrix, im_z, im_s, regionIndex, contractionType,
                                         propagation_dir, Detectmode=pg4_curr_detection_mode.get(),
                                         anterograde_tolerance_intercept=float(ati_in.get()),
                                         anterograde_tolerance_slope=int(ats_in.get()),
                                         retrograde_tolerance_intercept=float(rti_in.get()),
                                         retrograde_tolerance_slope=int(rts_in.get()), FigDisplay=True)

        file_list = [i for i in os.listdir(os.getcwd()) if f"{STmap_file_prefix}_region{regionIndex}" in i]
        for cur_fname in file_list:
            if "wavefront" not in cur_fname and "wavefrontSigmoidFit" not in cur_fname:
                continue
            image = load_image(cur_fname, int(canvas_x_dim - canvas_offset * 2), int(canvas_y_dim - canvas_offset * 2))
            frame4_gallery.append(image)

        if len(frame4_gallery):
            page4_bg = None
            beautify = [f'{k} : {v}\n' for k, v in proptemp.items()] + ['', '', '',
                                                                        'To View Plots, Use Right Button On Screen']
            feature_descriptions = '\n'.join(beautify)
            pg4_canvas.itemconfig(pg4_canvas_txt_id, text=feature_descriptions)
            corners = pg4_canvas.bbox(pg4_canvas_txt_id)
            bbox_height = corners[3] - corners[1]
            if bbox_height <= canvas_y_dim - canvas_offset * 2:
                pg4_canvas.itemconfig(pg4_canvas_image_id, image=canvas_bg_image)
            else:
                pg4_canvas.delete(pg4_canvas_image_id)
                total_height = bbox_height + 60
                page4_bg = ImageTk.PhotoImage(
                    Image.fromarray(canvas_bg_matrix).resize((canvas_x_dim, total_height), Image.Resampling.LANCZOS))
                pg4_canvas_image_id = pg4_canvas.create_image(canvas_offset, canvas_offset - (
                            total_height - (canvas_y_dim - canvas_offset * 2)) / 2,
                                                              image=page4_bg, anchor='nw')
                pg4_canvas.image = page4_bg
                pg4_canvas.itemconfig(pg4_canvas_txt_id, text=feature_descriptions)
                pg4_canvas.tag_raise(pg4_canvas_txt_id)

            frame4_current_index = 0
            pg4_label.config(text=f"{frame4_current_index + 1}/{len(frame4_gallery) + 1}", justify=tk.CENTER)
    except Exception as e:
        pg4_canvas.itemconfig(pg4_canvas_txt_id, text=f"Fail To Extract Features\n{e}")


def image_on_left():
    """This function handles left shift button.
    Show the image (if exist) in front of the current displaying image in page-4 canvas gallery (a python list).
    If no such image exists, the current displaying image remains on the screen.

    :return: None
    """
    global frame4_current_index
    if len(frame4_gallery) == 0:
        return
    frame4_current_index = max(frame4_current_index - 1, 0)
    if frame4_current_index == 0:
        if page4_bg is None:
            pg4_canvas.itemconfig(pg4_canvas_image_id, image=canvas_bg_image)
        else:
            pg4_canvas.itemconfig(pg4_canvas_image_id, image=page4_bg)
        pg4_canvas.itemconfig(pg4_canvas_txt_id, text=feature_descriptions)
    else:
        pg4_canvas.itemconfig(pg4_canvas_txt_id, text=f"")
        pg4_canvas.itemconfig(pg4_canvas_image_id, image=frame4_gallery[frame4_current_index - 1])
    pg4_label.config(text=f"{frame4_current_index + 1}/{len(frame4_gallery) + 1}", justify=tk.CENTER)


def image_on_right():
    """This function handles right shift button.
    Show the image (if exist) behind the current displaying image in page-4 canvas gallery (a python list).
    If no such image exists, the current displaying image remains on the screen.

    :return: None
    """
    global frame4_current_index
    frame4_current_index = min(frame4_current_index + 1, len(frame4_gallery))
    pg4_canvas.itemconfig(pg4_canvas_txt_id, text=f" ")
    pg4_label.config(text=f"{frame4_current_index + 1}/{len(frame4_gallery) + 1}", justify=tk.CENTER)
    pg4_canvas.itemconfig(pg4_canvas_image_id, image=frame4_gallery[frame4_current_index - 1])


def save_features_prop():
    """Save extracted propagation features to global var, for future saving to disk

    :return: None
    """
    global sample_feature_dict
    sample_feature_dict[f"region_{regionIndex}"] = proptemp


def go_back_to_page3():
    """Return to the previous page 3 propagation. Raise page 3 to top of the stack.

    :return: None
    """
    root.title("Propagation Analysis Step 1")
    pg3_prop_frame.lift()

## Page 4 Scripts

In [28]:
frame4_gallery = []
frame4_current_index = 0
feature_descriptions = ""
proptemp = ""
page4_bg = None

In [29]:
pg4_frame = ttk.Frame(root, width=dim[0], height=dim[1])
pg4_frame.grid(row=0, column=0, sticky=tk.N + tk.E + tk.S + tk.W)

pg4_canvas = Canvas(pg4_frame, bg='#FFFFFF', width=canvas_x_dim, height=canvas_y_dim)
pg4_canvas.grid(row=0, column=0, rowspan=10, columnspan=2)
pg4_canvas_image_id = pg4_canvas.create_image(canvas_offset, canvas_offset, image=canvas_bg_image, anchor='nw')
pg4_canvas_txt_id = pg4_canvas.create_text(canvas_x_dim // 2, canvas_y_dim // 2, width=canvas_x_dim,
                                           text="Wave Fronts Analysis Plots Go Here.",
                                           justify=tk.CENTER)

# Menu for Finer Propagation Type
pg4_contraction_menu_options = ['Propagation', 'Interrupted Small Contraction']
pg4_curr_contraction = StringVar()
pg4_curr_contraction.set("Propagation Direction Types")
pg4_contraction_dropdown = OptionMenu(pg4_frame, pg4_curr_contraction, *pg4_contraction_menu_options,
                                      command=update_contraction_type)
pg4_contraction_dropdown.grid(row=0, column=2, columnspan=2, sticky=tk.E + tk.W)

# Menu for Detection Mode Type
pg4_detection_menu_options = ['DBSCAN', 'neighborhood']
pg4_curr_detection_mode = StringVar()
pg4_curr_detection_mode.set("Detection Mode Types")
pg4_detection_dropdown = OptionMenu(pg4_frame, pg4_curr_detection_mode, *pg4_detection_menu_options)
pg4_detection_dropdown.grid(row=1, column=2, columnspan=2, sticky=tk.E + tk.W)

# Textbox for detection parameters
ati_in = ttk.Entry(pg4_frame)
ats_in = ttk.Entry(pg4_frame)
rti_in = ttk.Entry(pg4_frame)
rts_in = ttk.Entry(pg4_frame)

# Set default detection parameters
ati_in.insert(0, "0.5")
ats_in.insert(0, "5000")
rti_in.insert(0, "0.5")
rts_in.insert(0, "5000")

# Hint users with detection parameter fields
ttk.Label(pg4_frame, text="Anterograde Tolerance Intercept").grid(row=2, column=2, sticky=tk.E + tk.W)
ttk.Label(pg4_frame, text="Anterograde Tolerance Slope").grid(row=3, column=2, sticky=tk.E + tk.W)
ttk.Label(pg4_frame, text="Retrograde Tolerance Intercept").grid(row=4, column=2, sticky=tk.E + tk.W)
ttk.Label(pg4_frame, text="Retrograde Tolerance Slope").grid(row=5, column=2, sticky=tk.E + tk.W)

# Detection parameter layout
ati_in.grid(row=2, column=3, sticky=tk.E + tk.W)
ats_in.grid(row=3, column=3, sticky=tk.E + tk.W)
rti_in.grid(row=4, column=3, sticky=tk.E + tk.W)
rts_in.grid(row=5, column=3, sticky=tk.E + tk.W)

In [30]:
# Run Feature Extraction
pg4_ft_extract = ttk.Button(pg4_frame, text="Run Feature Extraction", command=run_feature_extraction)
pg4_ft_extract.grid(row=6, column=2, columnspan=2, sticky=tk.E + tk.W)

# Save features
pg4_save_ft = ttk.Button(pg4_frame, text="Save Extracted Features", command=save_features_prop)
pg4_save_ft.grid(row=7, column=2, columnspan=2, sticky=tk.E + tk.W)

# Label number
pg4_label = ttk.Label(pg4_frame, text=" ", justify=tk.CENTER, anchor=tk.CENTER)
pg4_label.grid(row=10, column=0, columnspan=2, sticky=tk.E + tk.W)

# View image/text descriptions on left
pg4_canvas_left = ttk.Button(pg4_frame, text="Left", command=image_on_left)
pg4_canvas_left.grid(row=11, column=0, sticky=tk.E + tk.W)

# View image/text descriptions on right
pg4_canvas_right = ttk.Button(pg4_frame, text="Right", command=image_on_right)
pg4_canvas_right.grid(row=11, column=1, sticky=tk.E + tk.W)

# Button to back to the page 3 propagation
pg4_prev = ttk.Button(pg4_frame, text="Back To Previous Page", command=go_back_to_page3)
pg4_prev.grid(row=8, column=2, columnspan=2, sticky=tk.E + tk.W)

# Button to the main analysis page
pg4_done = ttk.Button(pg4_frame, text="Finish And Go To Analysis Main Page", command=return_page2)
pg4_done.grid(row=9, column=2, columnspan=2, sticky=tk.E + tk.W)

pg4_frame.columnconfigure(2, weight=1)
pg4_frame.columnconfigure(3, weight=1)

In [31]:
pg4_canvas.bind("<MouseWheel>", on_mousewheel)

'2725933085312on_mousewheel'

# Infinite Loop (Main)

In [ ]:
pg1_frame.lift()
root.mainloop()

<class 'numpy.ndarray'>
The retrograde propagation start point position percentage is: 21.445221445221446 %
The retrograde propagation end point position percentage is: 0.0 %
The retrograde propagation distance for the contraction is: 1.1097708082026536 cm
The retrograde propagation distance percentage is: 21.445221445221442 %
The retrograde propagation time for the contraction is: 3.8 s
The overall retrograde propagation velocity for the contraction is: 0.2920449495270141 cm/s
The retrograde propagation mid point position percentage is: 19.451438860700407 %
The retrograde propagation mid point slope is: 6.994173887193751
Contraction region area: 0.06254174299845323 cm^2
Contraction region solidity:  0.723209590913871
Contraction centroid time position: 650.8860840482769 s, distance position percentage: 15.941062960111966 %
Maximum contraction intensity: -3.617893466432954
Raw diameter at maximum contraction intensity: 0.29508196721311475
Pixel coordinate of maximum contraction intensi